# Mettre au point un algorithme de siretisation d’une liste d’établissement rejetant des polluants dans l’air
## Premier pas avec le moteur de recherche ElasticSearch pour siretiser des établissements (Notebook en python)


### 1. Pourquoi Elasticsearch? 

**Elasticsearch** est un _moteur de recherche et d'analyse_ dans un ensemble de documents particulièrement performant sur les données textuelles, ou pour toute requête structurée comprenant des données numériques, textuelles, géospatiales. Les résultats de la recherche sont classés automatiquement par pertinence. Les recherches peuvent être hautement flexibles et bénéficier d'un utilisateur métier expert qui saurait comment la spécifier.  

Dans le cadre particulier de l'identification des entreprises, Elasticsearch fait partie de la solution retenue par
* l'API "Sirene données ouvertes" (DINUM) (cf https://annuaire-entreprises.data.gouv.fr/) 
* l'API de recherche d'entreprises Française de la Fabrique numérique des Ministères Sociaux (cf https://api.recherche-entreprises.fabrique.social.gouv.fr/)
* le projet de l'Insee "Amélioration de l'identification de l'employeur dans le recensement", pour faire une première sélection des établissements pertinents pour un bulletin de recensement donné. 

Dans le cadre de l'identification des individus, Elasticsearch fait partie de la solution envisagée pour l'identification des individus au RNIPP (Répertoire national des personnes physiques) pour le projet CSNS (Code statistique non signifiant), et est la solution technique sous-jacente au projet [matchID](https://matchid.io/) du ministère de l'intérieur.

C'est également un outil qui peut être utilisé pour des appariements flous ad-hoc dans le cadre d'étude à l'Insee, par exemple au niveau produits entre les données de caisse de RelevanC et OpenFoodFacts et entre les points de ventes RelevanC et Sirius [[Communication JMS](http://jms-insee.fr/jms2022s28_2/)].

Une introduction à Elastic Search pour l'appariement flou est disponible sur le datalab ici : [Notebook d'introduction](https://www.sspcloud.fr/formation?search=&path=%5B%22Analyse%20Textuelle%22%5D)

Au delà du secteur public, on peut citer qu'Amazon AWS fait partie des utilisateurs historiques d'Elasticsearch. 

### 2. Présentation générale et vocabulaire 


Un **index**  est une collection de **documents** dans lesquels on souhaite chercher, préalablement ingérés dans un moteur de recherche Elasticsearch (étape d'indexation), dans notre cas d'usage, les documents sont les établissements. **L'étape d'indexation a été faites préalablement dans un moteur mis à disposition sur le datalab à tous.**   L'indexation consiste à pré-réaliser les traitements des termes des documents pour gagner en efficacité lors de la phase de recherche. L'indexation est faites une fois pour de nombreuses recherches potentielles, pour lesquelles la rapidité de réponse peut être crutiale.

Les documents sont constitués de variables, les **champs** ('fields'), dont le **type** est spécifié ("text", "keywoard", "geo_point", "numeric"...) à l'indexation.

Les **analyzers** sont très utiles pour les données textuelles: ils se composent en général d'un **tokenizer** (méthode pour séparer le texte en éléments unitaires, les tokens, en général des mots, mais cela peut aussi être des n-grammes de caractères) et de **filtres**, par exemple de certains mots (stopwords, gestion des synonymes). Chaque champ peut être associé à un analyzer particulier, dans un objet défini à l'indexation, le **mapping**. Le mapping comporte le schéma des données ainsi que la façon dont ils seront analysés pour la recherche, i.e. champ, type et analyzer associé.

L'utilisateur va requêter le moteur de recherche via des **query**. Ces dernières sont très flexibles et constitue un langage en soi: on parle de Query DSL (Domain-Specific Language). 

La documentation Elastic va répondre à au moins trois enjeux, à savoir distinguer pour ne pas s'y perdre:

1. Mettre en place et configurer un cluster Elastic Search adapté au besoin 
2. Indexer intelligemment ses données: définir les types et les analyzers 
3. Requêter intelligemment ces données 

L'étape 1 a été gérée par la DIIT via la mise en place de services Elastic Search sur le datalab. L'étape 2 a été gérée par le SSP Lab, en reprenant pour le funathon l'indexation utilisée dans le cadre du projet AIEE "Amélioration de l'identification de l'employeur dans le recensement". Le sujet peut donc être traité intégralement en s'intéressant à la dernière étape. 

Les mots clés utiles pour parcourir la documentation sont "Query DSL", "Text analysis", "Search data". 
https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html 

Il peut être utile de parcourir le [Cours ensae Python](https://pythonds.linogaliana.fr/elastic/), bien qu'il y ait quelques redondances avec la suite de ce notebook sur la partie "requête", la partie indexation y est abordée.


### 3. Connexion au moteur Elastic mis en place pour le funathon 

ElasticSearch est installé sur un serveur (ici le datalab) qu'il est possible de requêter depuis un client, par exemple une session R ou Python du datalab, ou encore, l'interface graphique associée à ElasticSearch nommée Kibana. Cette dernière est pratique
pour tester des requêtes mais elle n'a pas été rendue disponible pour chaque participant à l'occasion du funathon. En revanche, pour aller plus loin, il est possible d'ouvrir son propre service ElasticSearch via le datalab (étape 1), d'y indexer ses propres données (étape 2), et d'avoir accès à l'interface Kibana. 


Nous allons utiliser la librairie `python` `elasticsearch` pour dialoguer avec notre moteur de recherche elastic. Les instructions ci dessous indiquent comment établir la connection.

In [1]:
! pip install elasticsearch==7.17.3 # Version compatible avec la version datalab. https://elasticsearch-py.readthedocs.io/en/v7.13.4/api.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 12.5 MB/s eta 0:00:00


In [1]:
from elasticsearch import Elasticsearch
HOST = 'elasticsearch-master.projet-ssplab'

def elastic():
    """Connection avec Elastic sur le data lab"""
    es = Elasticsearch([{'host': HOST, 'port': 9200, 'scheme': 'http'}], http_compress=True, request_timeout=200)
    return es

es = elastic()

**Moteur de recherche:** Le moteur de recherche proposé est une brique logicielle développée dans le cadre du projet AIEE (Amélioration de l'Identification de l'Etablissement Employeur dans les données collectées du RP). Il s'agit d'un moteur de recherche dans lequel les données de sirus 2020 sont mis à disposition dans un _index_, avec certains champs enrichis, comme la géolocalisation.


In [2]:
# On compte les établissements indexés dans le moteur de recherche #
es.count(index = "sirus_2020")

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'count': 11184109,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}}

**Données:** IREP Registre français des émissions polluantes (obligation de déclaration: https://www.legifrance.gouv.fr/loda/id/JORFTEXT000018276495/, description des données ici: https://www.georisques.gouv.fr/risques/registre-des-emissions-polluantes) 
Cette base contient des informations sur l'établissement, dont sa dénomination et sa localisation; ainsi que le siret, que l'on traitera ici comme une inconnue pour mettre au point l'algorithme de siretisation, et que l'on utilisera pour valider l'algorithme mis au point. 

In [3]:
# Importation des bases

import functions as fc

dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

dict_keys(['rejets', 'etablissements', 'emissions', 'Trait_dechets_non_dangereux', 'Trait_dechets_dangereux', 'Prod_dechets_non_dangereux', 'Prod_dechets_dangereux', 'Prelevements'])

In [4]:
###################
# --- Données --- #
###################

# On les récupère sur data.gouv.fr
# https://www.data.gouv.fr/en/datasets/registre-francais-des-emissions-polluantes/

import pandas as pd

df = dict_data["etablissements"]
df = df.drop('numero_siret', axis = 1) # Pour ne pas tricher

# Un exemple pour démarrer
df[df['identifiant']==6506233]

,identifiant,nom_etablissement,adresse,code_postal,commune,departement,region,coordonnees_x,coordonnees_y,code_epsg,code_ape,libelle_ape,code_eprtr,libelle_eprtr
7273,6506233,CPCU - CENTRALE DE BERCY,"177, rue de Bercy",75012,PARIS-12E-ARRONDISSEMENT,PARIS,ILE-DE-FRANCE,2.37396,48.84329,4326.0,3530Z,Production et distribution de vapeur et d'air ...,1.(c),Centrales thermiques et autres installations d...


Dans la suite, on va s'intéresser à siretiser un exemple, la `CPCU - Centrale de Bercy` - Elle déclare des rejets de CO2: 

In [5]:
em = dict_data["emissions"]
em[em['identifiant']==6506233]

,identifiant,nom_etablissement,annee_emission,milieu,polluant,quantite,unite
4550,6506233,CPCU - CENTRALE DE BERCY,2019,Air,Dioxyde de carbone (CO2) d'origine non biomasse,23800000.0,kg/an
4551,6506233,CPCU - CENTRALE DE BERCY,2019,Air,Dioxyde de carbone (CO2) total (d'origine biom...,30500000.0,kg/an


# Première recherche

In [6]:
# Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
#fullsearch = es.search(index = "sirus_2020", # l'index dans lequel on cherche
#                              q = "CPCU - CENTRALE DE BERCY", # notre requête textuelle
#                              size = 1) # taille de l'ensemble les échos souhaités
# debug? unstable behaviour?
fullsearch = es.search(index = "sirus_2020", # l'index dans lequel on cherche
                       q = "CPCU - CENTRALE DE BERCY", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Dans le résultat de la recherche, on obtient une liste avec 
* le temps que la requête a prise (`took`) 
* une liste des échos obtenus (`hits`), ici nous n'en avons demandé qu'un seul, mais il y a aussi le nombre total d'écho obtenu `fullsearch['hits']['total']['value']`

In [7]:
fullsearch

{'took': 330,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 9.0,
  'hits': [{'_index': 'sirus_2020_e_3_ngr_bool',
    '_type': '_doc',
    '_id': '85086568400011',
    '_score': 9.0,
    '_source': {'sirus_id': '850865684',
     'nic': '00011',
     'ape': '3511Z',
     'apet': '3511Z',
     'eff_3112_et': '',
     'eff_etp_et': '',
     'eff_et_effet_daaaammjj': '',
     'enseigne_et1': '',
     'nom_comm_et': '',
     'adr_et_loc_geo': '7511200221',
     'adr_et_compl': '173-175-TOUR LYON BERCY',
     'adr_et_voie_num': '173',
     'adr_et_voie_repet': '',
     'adr_et_voie_type': 'RUE',
     'adr_et_voie_lib': 'DE BERCY',
     'adr_et_cedex': '',
     'adr_et_distsp': '',
     'sir_adr_et_com_lib': 'PARIS 12',
     'adr_et_post': '75012',
     'adr_et_l1': 'SAS CENTRALE SOLAIRE ALBI PELISSIER',
     'adr_et_l2': '',
     'adr_et_l3': '173-175-TOUR LYON BERCY',
     'ad

In [20]:
# Les résultats sont restitués sous forme de listes de liste
fullsearch['hits']['hits'][0]['_source']

{'sirus_id': '850865684',
 'nic': '00011',
 'ape': '3511Z',
 'apet': '3511Z',
 'eff_3112_et': '',
 'eff_etp_et': '',
 'eff_et_effet_daaaammjj': '',
 'enseigne_et1': '',
 'nom_comm_et': '',
 'adr_et_loc_geo': '7511200221',
 'adr_et_compl': '173-175-TOUR LYON BERCY',
 'adr_et_voie_num': '173',
 'adr_et_voie_repet': '',
 'adr_et_voie_type': 'RUE',
 'adr_et_voie_lib': 'DE BERCY',
 'adr_et_cedex': '',
 'adr_et_distsp': '',
 'sir_adr_et_com_lib': 'PARIS 12',
 'adr_et_post': '75012',
 'adr_et_l1': 'SAS CENTRALE SOLAIRE ALBI PELISSIER',
 'adr_et_l2': '',
 'adr_et_l3': '173-175-TOUR LYON BERCY',
 'adr_et_l4': '173 RUE DE BERCY',
 'adr_et_l5': '',
 'adr_et_l6': '75012 PARIS 12',
 'adr_et_l7': '',
 'nic_siege': '00011',
 'unite_type': '1',
 'region': '11',
 'adr_depcom': '75112',
 'region_impl': '11',
 'region_mult': 'MULT',
 'tr_eff_etp': 'NN',
 'cj': '5710',
 'denom': 'SAS CENTRALE SOLAIRE ALBI PELISSIER',
 'denom_condense': 'SAS CENTRALE SOLAIRE ALBI PELISSIER',
 'sigle': '',
 'enseigne': '',


# Raffiner la recherche

Pour mieux cibler la requête, on voudrait exploiter un peu plus l'information dont on dispose sur l'établissement

On peut

1. Faire une requête "champ à champ", plutôt que de chercher dans l'ensemble des champ comme précedemment
2. Filtrer sur certains critères, par exemple l'apet ou les coordonnées géographiques




Pour cela, on va écrire une requête plus détaillée, au format json. 

Voici un exemple de requête où l'on souhaite définir la pertinence d'un écho sur la base de la similarité entre le champ indexé `rs_denom` et `CPCU - Centrale de Bercy` (clause `must`) et ne rechercher des échos que s'ils sont dans un rayon de 0.5 km des coordonnées géographiques de l'établissement recherché et qu'ils ont le bon apet (clause `filter`). Les conditions sous la clause `match` vont permettre la notation de la pertinence, les conditions sous la clause `filter` vont restreindre les recherches sans jouer sur la notation des échos.



In [21]:
specificsearch = es.search(index = 'sirus_2020', body = 
'''{
  "query": {
    "bool": {
      "should":
          { "match": { "rs_denom":   "CPCU - CENTRALE DE BERCY"}},
      "filter": [
          {"geo_distance": {
                  "distance": "0.5km",
                  "location": {
                        "lat": "48.84329", 
                        "lon": "2.37396"
                              }
                            }
            }, 
            { "prefix":  { "apet": "3530" }}
                ]
            }
          }
}'''
)

/tmp/ipykernel_192/2974778146.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  specificsearch = es.search(index = 'sirus_2020', body =


In [22]:
specificsearch['hits']['hits'][0]['_source']

{'sirus_id': '542097324',
 'nic': '00066',
 'ape': '3530Z',
 'apet': '3530Z',
 'eff_3112_et': '114.0',
 'eff_etp_et': '110.0',
 'eff_et_effet_daaaammjj': '20181231',
 'enseigne_et1': '',
 'nom_comm_et': '',
 'adr_et_loc_geo': '7511200221',
 'adr_et_compl': '',
 'adr_et_voie_num': '177',
 'adr_et_voie_repet': '',
 'adr_et_voie_type': 'RUE',
 'adr_et_voie_lib': 'DE BERCY',
 'adr_et_cedex': '',
 'adr_et_distsp': '',
 'sir_adr_et_com_lib': 'PARIS 12',
 'adr_et_post': '75012',
 'adr_et_l1': 'CIE PARISIENNE DE CHAUFFAGE URBAIN',
 'adr_et_l2': '',
 'adr_et_l3': '',
 'adr_et_l4': '177 RUE DE BERCY',
 'adr_et_l5': '',
 'adr_et_l6': '75012 PARIS 12',
 'adr_et_l7': '',
 'nic_siege': '00017',
 'unite_type': '1',
 'region': '11',
 'adr_depcom': '75112',
 'region_impl': '11',
 'region_mult': 'QASI',
 'tr_eff_etp': '41',
 'cj': '5599',
 'denom': 'COMPAGNIE PARISIENNE DE CHAUFFAGE URBAIN',
 'denom_condense': 'CIE PARISIENNE DE CHAUFFAGE URBAIN',
 'sigle': 'CPCU',
 'enseigne': '',
 'eff_3112_unitelegal

Tentons un autre établissement, avec un autre exemple de requête

In [23]:
df[df['identifiant']==7404213]

,identifiant,nom_etablissement,adresse,code_postal,commune,departement,region,coordonnees_x,coordonnees_y,code_epsg,code_ape,libelle_ape,code_eprtr,libelle_eprtr
7804,7404213,AP-HP SERVICE CENTRAL DES BLANCHISSERIES,139 Boulevard Macdonald,75019,PARIS-19E_-RRONDISSEMENT,PARIS,ILE-DE-FRANCE,654551.98,6866696.11,2154.0,9601A,Blanchisserie-teinturerie de gros,NaN,NaN


In [24]:
specificsearch = es.search(index = 'sirus_2020', body =
'''{
  "query": {
    "bool": {
      "should": [
          { "match": { "rs_denom":   "AP-HP SERVICE CENTRAL DES BLANCHISSERIES"}},
          { "match": { "geo_adresse":   "139 Boulevard Macdonald 75019 Paris"}}
        ],
      "filter":
            { "prefix":  { "apet": "9601" }}
            }
          }
}'''
)

/tmp/ipykernel_192/1357246710.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  specificsearch = es.search(index = 'sirus_2020', body =
/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)




Les requêtes sont très flexibles. Il faut se familiariser avec la documentation elastic (en anglais..): https://www.elastic.co/guide/en/elasticsearch/reference/7.0/query-dsl.html
Quand on maitrise celle-ci, on peut très efficacement adapter le requêtage à notre besoin. Mais ce n'est pas presse bouton !

Ne pas hésiter à se familiariser avec des exemples connus : 

In [25]:
# Exemple 1 (filtre dans un rayon de 10km autour d'un point, on cherche une denomination).
ex1 = es.search(index = 'sirus_2020', body = '''{
  "query": {
    "bool": {
      "must":
      { "match": { "denom":   "institut national de la statistique"}}
      ,
      "filter":
        {"geo_distance": {
          "distance": "10km",
          "location": {
            "lat": "48.8168",
            "lon": "2.3099"
          }
        }
      }
    }
  }
}
''')['hits']['hits']
ex1

/tmp/ipykernel_192/128728356.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  ex1 = es.search(index = 'sirus_2020', body = '''{


[{'_index': 'sirus_2020_e_3_ngr_bool',
  '_type': '_doc',
  '_id': '12002701600019',
  '_score': 5.0,
  '_source': {'sirus_id': '120027016',
   'nic': '00019',
   'ape': '8411Z',
   'apet': '8411Z',
   'eff_3112_et': '1360.0',
   'eff_etp_et': '1274.0',
   'eff_et_effet_daaaammjj': '20181231',
   'enseigne_et1': 'DIRECTION GENERALE',
   'nom_comm_et': '',
   'adr_et_loc_geo': '7511400057',
   'adr_et_compl': '',
   'adr_et_voie_num': '18',
   'adr_et_voie_repet': '',
   'adr_et_voie_type': 'BD',
   'adr_et_voie_lib': 'ADOLPHE PINARD',
   'adr_et_cedex': '75675',
   'adr_et_distsp': '',
   'sir_adr_et_com_lib': 'PARIS CEDEX 14',
   'adr_et_post': '75014',
   'adr_et_l1': 'INS NAT STATISTIQUE ETUDES ECONOMIQUES',
   'adr_et_l2': 'DIRECTION GENERALE',
   'adr_et_l3': '',
   'adr_et_l4': '18 BD ADOLPHE PINARD',
   'adr_et_l5': 'PARIS 14',
   'adr_et_l6': '75675 PARIS CEDEX 14',
   'adr_et_l7': '',
   'nic_siege': '00563',
   'unite_type': '1',
   'region': '11',
   'adr_depcom': '92049',
 

In [26]:
# Exemple 2 (On cherche un mot clé dans plusieurs champs, dans une commune, en filtrant sur les apet du commerce alimentaire)
ex2 = es.search(index = 'sirus_2020', body = '''{
  "query": {
    "bool": {
      "should":
      [
        {
            "multi_match" : {
                      "query":      "FRANPRIX",
                      "type":       "best_fields",
                      "fields":     [ "denom", "enseigne", "nom_comm_et", "adr_et_l1","adr_et_l2", "denom_condense", "enseigne_et1" ],
                      "tie_breaker": 0.1
                      }
        },
        { "match": { "sir_adr_et_com_lib": "montrouge" }}
      ],
      "minimum_should_match": 2,
      "filter": [
        { "prefix":  { "apet": "4711" }}
      ]
    }
  }
}
''')['hits']['hits']
ex2

/tmp/ipykernel_192/2088976289.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  ex2 = es.search(index = 'sirus_2020', body = '''{


[{'_index': 'sirus_2020_e_3_ngr_bool',
  '_type': '_doc',
  '_id': '80494939400030',
  '_score': 2.2,
  '_source': {'sirus_id': '804949394',
   'nic': '00030',
   'ape': '4711D',
   'apet': '4711D',
   'eff_3112_et': '10.0',
   'eff_etp_et': '6.0',
   'eff_et_effet_daaaammjj': '20181231',
   'enseigne_et1': 'FRANPRIX',
   'nom_comm_et': '',
   'adr_et_loc_geo': '9204900031',
   'adr_et_compl': '4-6',
   'adr_et_voie_num': '4',
   'adr_et_voie_repet': '',
   'adr_et_voie_type': 'RUE',
   'adr_et_voie_lib': 'DANTON',
   'adr_et_cedex': '',
   'adr_et_distsp': '',
   'sir_adr_et_com_lib': 'MONTROUGE',
   'adr_et_post': '92120',
   'adr_et_l1': 'BAREDIS',
   'adr_et_l2': 'FRANPRIX',
   'adr_et_l3': '4-6',
   'adr_et_l4': '4 RUE DANTON',
   'adr_et_l5': '',
   'adr_et_l6': '92120 MONTROUGE',
   'adr_et_l7': '',
   'nic_siege': '00030',
   'unite_type': '1',
   'region': '11',
   'adr_depcom': '92049',
   'region_impl': '11',
   'region_mult': 'MONO',
   'tr_eff_etp': '03',
   'cj': '5202',


# Explorer les champs et tester des requêtes


Toutes les requêtes ne fonctionneront pas forcément pour tous les cas. Pour créer des requêtes adaptées pour le cas d'usage envisagé, il faut un peu mieux connaître l'index : quels sont les champs disponibles, comment sont-ils spécifiés et analysés? 

Ces informations sont disponibles dans le "mapping" de l'index: 



In [27]:
es.indices.get_mapping(index = 'sirus_2020')

{'sirus_2020_e_3_ngr_bool': {'mappings': {'properties': {'adr_depcom': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_cedex': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_compl': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_distsp': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_l1': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256},
      'ngr': {'type': 'text', 'analyzer': 'ngram_analyzer'},
      'stem': {'type': 'text', 'analyzer': 'stemming'}},
     'copy_to': ['fourretout', 'rs_adr'],
     'norms': False},
    'adr_et_l2': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256},
      'ngr': {'type': 'text', 'analyzer': 'ngram_analyzer'}},
     'copy_to': ['fourretout', 'rs_adr'],
     'norms': False},
  

Le mapping choisi pour le projet décrit les champs indexés (variables spécifiques à l'établissement qui sont accessibles), la façon dont ils seront analysés par une requête

In [28]:
# Nom des champs 
mapping = es.indices.get_mapping(index = 'sirus_2020')
[s for s in mapping['sirus_2020_e_3_ngr_bool']['mappings']['properties']]

['adr_depcom',
 'adr_et_cedex',
 'adr_et_compl',
 'adr_et_distsp',
 'adr_et_l1',
 'adr_et_l2',
 'adr_et_l3',
 'adr_et_l4',
 'adr_et_l5',
 'adr_et_l6',
 'adr_et_l7',
 'adr_et_loc_geo',
 'adr_et_post',
 'adr_et_voie_lib',
 'adr_et_voie_num',
 'adr_et_voie_repet',
 'adr_et_voie_type',
 'ape',
 'apet',
 'cj',
 'creat_daaaammjj',
 'denom',
 'denom_condense',
 'eff_3112_et',
 'eff_3112_uniteLegale',
 'eff_3112_unitelegale',
 'eff_effet_daaaammjj_uniteLegale',
 'eff_effet_daaaammjj_unitelegale',
 'eff_et_effet_daaaammjj',
 'eff_etp_et',
 'eff_etp_uniteLegale',
 'eff_etp_unitelegale',
 'enseigne',
 'enseigne_et1',
 'fourretout',
 'geo_adresse',
 'geo_score',
 'geo_type',
 'latitude',
 'location',
 'longitude',
 'nic',
 'nic_siege',
 'nom_comm_et',
 'region',
 'region_impl',
 'region_mult',
 'rs_adr',
 'rs_denom',
 'sigle',
 'sigle_denom',
 'sigle_l1',
 'sir_adr_et_com_lib',
 'siret',
 'siret_id',
 'sirus_id',
 'tr_eff_etp',
 'unite_type']

Ci dessous, on voit que le champ "enseigne" est considéré de deux façons: en tant que `keyword` il est analysé exactement, et ne pourra matcher qu'avec un champ contenant les mêmes mots; en tant que `ngr`(n-grammes), il est analysé à travers les sous ensembles de n caractères, et donc pourra matcher avec un champ similaire à des fautes de frappes près. Il est recopié dans un champ `rs_denom`, champ que nous avons utilisé dans les requêtes précédentes, un champ "fourre tout".

In [29]:
mapping['sirus_2020_e_3_ngr_bool']['mappings']['properties']['enseigne']

{'type': 'text',
 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256},
  'ngr': {'type': 'text', 'analyzer': 'ngram_analyzer'}},
 'copy_to': ['fourretout', 'rs_denom']}

Le champ "location" est de type point géographique, et peut donc être analysé à travers des critères de distance.

In [30]:
mapping['sirus_2020_e_3_ngr_bool']['mappings']['properties']['location']

{'type': 'geo_point', 'ignore_malformed': True}

Il faut choisir les champs pertinent à requêter à partir de nos données.

Ensuite, il faut spécifier des modèles de requêtes, où l'on peut injecter automatiquement les champs requêtés, par exemple:


In [31]:
requete_type = '''{
  "query": {
    "bool": {
      "should": [
        { "match": { "rs_denom":   "{{nom_etablissement}}" }},
        { "match": { "geo_adresse": "{{adresse}}" }},
        { "match": { "sir_adr_et_com_lib": "{{commune}}" }}
      ],
      "minimum_should_match": 2,
      "filter": [
        { "match":  { "adr_et_post": "{{code_postal}}" }}
      ]
    }
  }
}'''

# Industrialiser les requêtes

Enfin, la fonction `msearch` de la librairie `elasticsearch` permet de passer plusieurs requêtes en parallèle (référence: https://elasticsearch-py.readthedocs.io/en/v7.13.4/)

On part d'un template de requête, que l'on souhaite utiliser pour une liste d'établissements à siretiser.

In [32]:
# On double les accolades pour utiliser la fonction format_map, qui utilise les balises {nom_var} pour remplir les champs
# On ne demande qu'un seul match par requête

requete_type = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{ "match": {{ "rs_denom":   "{nom_etablissement}" }}}},
        {{ "match": {{ "geo_adresse": "{adresse}" }}}},
        {{ "match": {{ "sir_adr_et_com_lib": "{commune}" }}}}
      ],
      "minimum_should_match": 2,
      "filter": [
        {{ "match":  {{ "adr_et_post": "{code_postal}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''

On peut ainsi peupler la requête type avec des champs particuliers:

In [33]:
requete_type.format_map({'nom_etablissement': 'STEP - BEZIERS', 'adresse': 'Plaine Saint Pierre','code_postal': 34500, 'commune': 'BEZIERS'})

'{ \n  "query": {\n    "bool": {\n      "should": [\n        { "match": { "rs_denom":   "STEP - BEZIERS" }},\n        { "match": { "geo_adresse": "Plaine Saint Pierre" }},\n        { "match": { "sir_adr_et_com_lib": "BEZIERS" }}\n      ],\n      "minimum_should_match": 2,\n      "filter": [\n        { "match":  { "adr_et_post": "34500" }}\n      ]\n    }\n  },\n  "size": 1\n}'

In [34]:
# Il est nécessaire de spécifier l'index associé à chaque requête
header = '{"index" : "sirus_2020"}'

multiple_requetes = ""

# On itère sur le dataframe d'établissements polluants pour ajouter une requête spécifique à chacun d'entre eux
n_etab = 10 # Pour l'exemple, on prend les 10 premiers

for index, row in df.iloc[0:n_etab][['nom_etablissement', 'adresse', 'code_postal', 'commune']].iterrows():
    
    multiple_requetes+= header
    multiple_requetes+= '\n'
    multiple_requetes+= requete_type.format_map(row).replace("\n","")
    multiple_requetes+= '\n'
    
# On colle ainsi au format de la requête acceptée par l'API multisearch: https://www.elastic.co/guide/en/elasticsearch/reference/7.13/search-multi-search.html 

In [35]:
res = es.msearch(body = multiple_requetes)

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [36]:
len(res['responses']) # Autant que d'établissements recherchés

10

In [37]:
res['responses'][0]['hits']['hits']

[{'_index': 'sirus_2020_e_3_ngr_bool',
  '_type': '_doc',
  '_id': '81162101000024',
  '_score': 5.0,
  '_source': {'sirus_id': '811621010',
   'nic': '00024',
   'ape': '8010Z',
   'apet': '8010Z',
   'eff_3112_et': '3.0',
   'eff_etp_et': '2.0',
   'eff_et_effet_daaaammjj': '20181231',
   'enseigne_et1': '',
   'nom_comm_et': '',
   'adr_et_loc_geo': '3403200276',
   'adr_et_compl': '',
   'adr_et_voie_num': '15',
   'adr_et_voie_repet': '',
   'adr_et_voie_type': '',
   'adr_et_voie_lib': 'PLAINE SAINT PIERRE',
   'adr_et_cedex': '',
   'adr_et_distsp': '',
   'sir_adr_et_com_lib': 'BEZIERS',
   'adr_et_post': '34500',
   'adr_et_l1': 'ALTEA SECURITE BEZIERS',
   'adr_et_l2': '',
   'adr_et_l3': '',
   'adr_et_l4': '15 PLAINE SAINT PIERRE',
   'adr_et_l5': '',
   'adr_et_l6': '34500 BEZIERS',
   'adr_et_l7': '',
   'nic_siege': '00024',
   'unite_type': '1',
   'region': '76',
   'adr_depcom': '34032',
   'region_impl': '76',
   'region_mult': 'QASI',
   'tr_eff_etp': '01',
   'cj':

Voilà, on a fait le tour des briques de bases pour démarrer: 

1. Tester des requêtes pour en définir quelques unes qui fonctionnent bien: 
explorer la documentation -> https://www.elastic.co/guide/en/elasticsearch/reference/7.0/query-dsl.html 

2. Industrialiser en faisant des requêtes multiples: fonction `msearch`